In [ ]:
!pip3 install pdf2image #installing the necessary package
!brew install poppler #turn this on for macs
!pip3 install cmake
!pip3 install python-poppler
!pip3 install bs4 lxml #this is required for xml parsing
!pip3 install natsort #for sorting files
!pip3 install opencv-python #open cv for converting images
!pip install pytesseract #installing pytesseract

In [ ]:
sample_pdf='' #this is the pdf with links
xml_main='' #this contains the page locations
annot_xml='' #this contains annotations width

out_pdf=""

import subprocess
import os

def remove_pdf_links(inp_pdf,out_pdf=None):
    
    
    if(out_pdf==None):
        out_pdf=inp_pdf.rsplit(".",1)
        out_pdf_temp=out_pdf[0]+"_stripped_temp."+out_pdf[1]
        out_pdf=out_pdf[0]+"_stripped."+out_pdf[1]
        
    
    
    #code for conversion
    first_cmd=" ".join(["pdftk", "{}".format(inp_pdf),"output", "{}".format(out_pdf_temp), "uncompress"])
    second_cmd="LC_ALL=C sed -n '/^\/Annots/!p' {} > {}".format(out_pdf_temp,inp_pdf)
    
    #call first command 
    
    subprocess.call(first_cmd,shell=True)
    
    #call second command
    ret=subprocess.call(second_cmd,shell=True)
    
    os.remove(out_pdf_temp)

remove_pdf_links(sample_pdf)

In [ ]:
from pdf2image import convert_from_path
from math import floor
from bs4 import BeautifulSoup as bs
import os
import cv2
from tqdm import tqdm


def pdf2_image(pdf_path,xml_main):
    
    #get the resolution of each pdf page from xml
    content = []
    # Read the XML file
    with open(xml_main, "r") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "xml")
        #print(bs_content)
        page_info=bs_content.find_all("Page")
        resolution=[]
        for page in page_info:
            height=floor(float(page.get("HEIGHT")))
            width=floor(float(page.get("WIDTH")))
            resolution.append((width,height))
    file.close()
    
    #print(resolution)
        
    #convert the pdf to image using the coordinates from the xml_file
    all_pages=[]
    
    ff=1
    lf=len(resolution)
    image = convert_from_path(pdf_path,first_page=ff,last_page=lf,use_pdftocairo=True,fmt="png")
    
    skip=False #default value
    
    if(len(resolution)!=len(image)):
        skip=True
        resolution.pop(0)
    
        
    #print(all_pages)
        
    #make a new directory in pdf folder
    
    return image,resolution,skip



#contains the list of the images
all_images,resolution,skip=pdf2_image(sample_pdf,xml_main)


print(len(all_images),len(resolution))

In [ ]:
#let's view the image and save
def get_magnifications(sample_pdf,resolution,all_images):
    
    try:
        path=sample_pdf.rsplit("/",1)[0]+"/images"
        #print(path)
        os.mkdir(path)
    except:
        print("path already exists !!!!")

    #print(path)     
    magnifications=[]
    
    i=0
    for image,res in zip(all_images,resolution):
        #print (image.format, image.size, image.mode)
        new_width=image.size[0]
        original_width=res[0]
        new_height=image.size[1]
        original_height=res[1]
        mag_width=new_width/original_width
        mag_height=new_height/original_height
        magnifications.append([mag_width,mag_height])
        #image.show()
        image.save(path+"/"+"image_"+str(i+1)+'.png', 'PNG')
        i+=1
        
    return magnifications

magnifications=get_magnifications(sample_pdf,resolution,all_images)

In [ ]:
print(magnifications)
print(len(magnifications))

# given the annot.xml file locate the coordinates of the boxes

In [ ]:
from math import floor
def get_all_boxes(xml,magnifications,skip=False):
    # Read the XML file
    with open(xml, "r") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        #print(content)
        content = "".join(content)
        bs_content = bs(content, "xml")
        
        
        page_info=bs_content.find_all("ANNOTATION")
        quads=bs_content.find_all("QUADRILATERAL")
        
        
        
        extractions=[]
        #filter magnifications for certain pages
        
        
        for action_,quad in zip(page_info,quads):
            if(action_.DEST.string!=None): #valid box
                page_no=int(action_.get("pagenum")) #basically the page_num
                
                class_=action_.DEST.string #the type of the box
                
                #if the type of the box is some link then skip it dont extract it
                if("@" in class_):
                    continue
                
                
                coord=[[floor(float(x.get("HPOS"))),floor(float(x.get("VPOS")))] for x in quad] #the coordinates
                if(skip):
                    extractions.append([page_no-1,coord,class_]) 
                else:
                    extractions.append([page_no,coord,class_]) 
    
    file.close()
    
    #print(extractions)
    #we need to filter magnifications for each page
    for i in range(len(magnifications)):
        #print(i)
        for k,elements in enumerate(extractions):
            #print(elements)
            #print(elements)
            if(elements[0]==i+1):
                
                #print(extractions[k][1])
                
                #change the two coordinates
                for d in range(len(elements[1])): #for all coordinates change their pix values
                    extractions[k][1][d][0]=floor(extractions[k][1][d][0]*magnifications[i][0])
                    extractions[k][1][d][1]=floor(extractions[k][1][d][1]*magnifications[i][1])
                
    return extractions

boxes=get_all_boxes(annot_xml,magnifications,skip)
print(len(boxes))

In [ ]:
print(boxes[:5]) #an example of boxes

# extract the information on the images from the pdfs

In [ ]:
import natsort 
from PIL import Image

def draw_rectangles(images_dir,boxes,show_images=False):
    """takes the images from the image path reads them and plots a rectange on them
    also return the patches for each detected page  
    """
    #print(images_dir)
    pages=[]
    box=[]
    names=[]
    
    boxes_per_page={}
    for info in boxes:
        page=info[0]
        rect=info[1]
        if(page not in boxes_per_page):
            boxes_per_page[page]=[rect]
        else:
            boxes_per_page[page].append(rect)
            
    #print(boxes_per_page)
    patches={}
    
    for k,v in boxes_per_page.items():
        image_path=images_dir+f"/image_{k}.png"
        image=cv2.imread(image_path)
        
        
        #v=[[1,2,3,4],[1,2,3,4]]
        cropped_images=[]
        
        #crop the images
        for boxes in v:
            #print(boxes)
            top_left=boxes[1] #x1,y1
            bottom_right=boxes[2] #x2,y2
            
            
            #cropping images
            cropped = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
             

            cropped_images.append(cropped)
                
            
        patches[k]= cropped_images
        
        if(show_images): #shows all boxes including filtered boxes
            for boxes in v:
                (x1,y1),(x2,y2)=boxes[1],boxes[2]
                image=cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 1)
                cv2.imshow("test",image)
                cv2.waitKey()
            
    return patches,boxes_per_page
            
        
path=sample_pdf.rsplit("/",1)[0]+"/images"  
patches,boxes_per_page=draw_rectangles(path,boxes,show_images=True) #patches only contain the protions of the 
print(boxes_per_page.keys())
print(patches.keys())

In [ ]:
#images before deletion
a=os.listdir(path)
a.sort()
print(len(a))

print(len(patches))
patch_len=0
for patch in patches.values():
    patch_len+=len(patch)

In [ ]:
def delete_irrelevant_images(patches,all_images,path):
    
    for i in range(1,len(all_images)): #from image 1 to last
        if(i not in patches.keys()):
            os.remove(path+"/"+"image_"+str(i)+'.png')
            
delete_irrelevant_images(patches,all_images,path)


In [ ]:
print(xml_main,len(magnifications),len(boxes))

In [ ]:
print(path) #you can check this path to verify

In [ ]:
a=os.listdir(path)
a.sort()
print(len(a))

# How can we filter ?

# The pytesseract way -(a) 

In [ ]:
#print(patches)

#check pytesseract for each image

try:
    from PIL import Image
except ImportError:
    import Image

from pytesseract import image_to_string

def image_to_text(image,show_image=False,show_text=False):

    # By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
    # we need to convert from BGR to RGB format/mode:
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ocr_chars=image_to_string(img_rgb)

    if(show_text):
        print("tesseract values >>>>>>>")
        print("this is my length: ",len(ocr_chars))
        print(ocr_chars)
        print("tesseract values <<<<<<<")
        print("\n")


    if(show_image):
        cv2.imshow("detection :",numpy_arr_of_image)
        cv2.waitKey()
        cv2.destroyAllWindows()
            
    return ocr_chars    
            
        
    #print(pytesseract.image_to_string(Image.open(patch)))

    


all_text=[]
for patch in patches.values():
    for numpy_arr_of_image in tqdm(patch):
        text=image_to_text(numpy_arr_of_image,show_image=False,show_text=False)
        all_text.append(text)
        
print("these are my all texts : ",len(all_text))

# The contour way of determining the patches - method (b)

In [ ]:
def get_contours_of_image(image,show_text=False,show_image=False):
               
    #we need to convert the image into gray scale to get contour values
    # Grayscale
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    gray = cv2.cvtColor(rgb_image,cv2.COLOR_BGR2GRAY)

    # Find Canny edges
    edged = cv2.Canny(gray, 30, 200)

    #print("contours: >>>>>>>")
    contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if(show_text):
        print("Number of Contours found = " + str(len(contours)))
    
    
    
    if(show_image):
        # Draw all contours
        # Use '-1' as the 3rd parameter to draw all
        cv2.drawContours(image, contours, -1, (0,255,0), 1)
        cv2.imshow('Contours:'+str(len(contours)), image)
        cv2.waitKey()
        cv2.destroyAllWindows()
                           
    return len(contours)
    #print(pytesseract.image_to_string(Image.open(patch)))
    


all_contours=[]
for patch in patches.values():
    for numpy_arr_of_image in patch:
        contours_count=get_contours_of_image(numpy_arr_of_image,show_text=False)
        all_contours.append(contours_count)
        #print("countours count: ",str(contours_count))
        
print("these are my all contours : ",len(all_contours))

# filter elements based on their area method -(c)

In [ ]:
def image_to_area(image,show_text=False):
    width=image.shape[0]
    height=image.shape[1]
    area=(width*height)/100 #in pixel square
    
    if(show_text):
        print(area)
        
    return area

all_areas=[]
for patch in patches.values():
    for numpy_arr_of_image in patch:
        area_image=image_to_area(numpy_arr_of_image,show_text=False)
        all_areas.append(area_image)
print("these are my all areas : ",len(all_areas))

# we need to get the text from xml file method -(d)

1. for all boxes in the boxes detected 
2. for all characters in the pdf- specifically that page
3. we need to do a comparison if the coordinates they lie on or under the theorem box
4. we need to concatenate each of the characters in the box for the given theorem box
5. we also take care of the spaces in between


In [ ]:
import copy
def box_to_text_using_xml(xml_main, magnifications,boxes,show_text=False,skip=False):
    """take a box coodinates xml file to check and adjusted magnifications
    and return what characters are inside the box basically text version
    """
    
    with open(xml_main,"r") as file:
        content=file.readlines()
        content = "".join(content)
        bs_content = bs(content, "xml")
        #print(bs_content)
        text_blocks={}
        page_info=bs_content.find_all("Page")
        
        if(skip):
            page_info.pop(0)
        
        
        for i,page in enumerate(page_info): #traverse through all pages
            
            
                
            page_text=page.find_all("TextLine")
            text=[]
            for text_bl in page_text:
    
                for element in text_bl:
                    if(str(element).startswith("<String")): #its a string 
                        try:
                            content=element.get("CONTENT")
                            #print(content)
                            h_pos=floor(float(element.get("HPOS"))*magnifications[i][0])
                            v_pos=floor(float(element.get("VPOS"))*magnifications[i][1])
                            #we can also extract the font 

                            text.append([content,h_pos,v_pos])
                        except:
                            print(element)
                            print(i)
                            print(magnifications[i][0])
                            print(content)
                        
                        
                    if(str(element).startswith("<SP")):
                        val=text.pop()
                        val[0]+=" "
                        text.append(val)
                        
                #print(text)
            
            text_blocks[i+1]=text
    
        file.close()
        
    
    
    box_text={}
    string_cats=[]
    for box in boxes: #basically the big box
        all_chars_on_page=text_blocks[box[0]] #small boxes

        #coordinates of the big box
        top_left_bb= copy.deepcopy(box[1][1])
        bot_right_bb=copy.deepcopy(box[1][2])


        #print(all_chars_on_page)
        string_cat=""
        
        for chars in all_chars_on_page:
            if(top_left_bb[0]<=chars[1]<=bot_right_bb[0] and top_left_bb[1]<=chars[2]<=bot_right_bb[1]):
                string_cat+=chars[0]
                    
        string_cats.append((box[0],string_cat))

    for element in string_cats:
        if(element[0] not in box_text):
            box_text[element[0]]=[element[1]]
        else:
            box_text[element[0]].append(element[1])
            
    return box_text
    
    
boxes_text=box_to_text_using_xml(xml_main,magnifications,boxes,show_text=False,skip=True)

print(len(boxes_text))

In [ ]:
print(xml_main,len(magnifications),len(boxes))

In [ ]:
print(len(boxes_text[3])) #21 text found on page 3

In [ ]:
#group each of the filtering technique together
def filter_thresholds(patches,boxes_text,boxes,show_text=False,show_image=False):
    #cleaning the mess
    area_th=2 #small blocks have low area
    contour_th=8 #plain horizontal lines have low contours
    text_th_ocr=2 #page number boxes have have low text
    text_th_xml=3

    thresholds=[text_th_ocr,contour_th,area_th,text_th_xml]
    
    box_map_cc=[]
    selection={}
    for page_info in patches.keys():

        verdicts=[]

        for image,text_xml in zip(patches[page_info],boxes_text[page_info]):

            area=image_to_area(image,show_text=False)
            contours_count=get_contours_of_image(image,show_text=False)
            #text_ocr=image_to_text(numpy_arr_of_image,show_image=False,show_text=False)

            if(show_text):
                print("image area: ",area)
                print("contours count: ",contours_count)
                #print("text using ocr: and length",(text_ocr,len(text_ocr)))
                print("text using xml: and length",(text_xml,len(text_xml)))
                print("=======")

            if(show_image):
                name="area {}. contours {}. text {}.".format(area,contours_count,text_xml)
                cv2.imshow("name",image)
                cv2.waitKey()
                cv2.destroyAllWindows()

            small_area= area<area_th
            small_contour_count= contours_count<contour_th

            #less_text_ocr= len(text_ocr)<text_th_ocr #using tesseract ocr
            less_text_xml=len(text_xml)<text_th_xml

            less_text=less_text_xml

            verdict=not(small_area or small_contour_count or less_text)
            verdicts.append(verdict)
            box_map_cc.append(verdict)
            

        selection[page_info]=verdicts


    from itertools import compress

    selection_=copy.deepcopy(selection)
    
    
    for k,sels,box in zip(selection_.copy(),selection_.values(),boxes_text.values()):
        new_images=list(compress(box, sels))

        if(len(new_images)==0):
            del selection[k]
        else:
            selection[k]=new_images
            

    #this is where we will start to filter each of the images
    box_map_cc=list(compress(boxes, box_map_cc))
    
    return selection ,box_map_cc     



selected,box_cc=filter_thresholds(patches,boxes_text,boxes,show_text=False,show_image=False)

new_patches,boxes_per_page=draw_rectangles(path,box_cc,show_images=False)


area_th=2 #small blocks have low area
contour_th=2 #plain horizontal lines have low conoturs
text_th_ocr=2 #page number boxes have have low text
text_th_xml=3

thresholds=[text_th_ocr,contour_th,area_th,text_th_xml]
selected=filter_thresholds(thresholds,patches,boxes_text,show_text=True,show_image=False)
new_patches,boxes_per_page=draw_rectangles(path,selected,show_images=True)

#in page 17 we can see that there are some objects that can be seen 

for patch in patches[17]:
    #show just boxes 
    cv2.imshow("name",patch)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    area=image_to_area(patch,show_text=False)
    contours_count=get_contours_of_image(patch,show_text=False)
    text=len(image_to_text(patch,show_image=False,show_text=False))

    print("image area: ",area)
    print("contours count: ",contours_count)
    print("text: ",text)
    print("=======")

In [ ]:
# original categories #these are the seven categories that are orignally intended for
def get_original_labels():
    """basically assigning labesls"""
    categories=["assumption","claim","conjecture","corollary","definition","lemma","proof","proposition",
                "remark", "theorem"]

    category_map={}

    for i,element in enumerate(categories):
        category_map[element]=i

    return category_map

category_map=get_original_labels()

#category map is a look up table 
print(category_map)



In [ ]:
#extract the data from the list and segment them according to the class
def make_lookups(selected,category_map,binary=True):
    look_up={} #this is the orgnal_lookup table matching page vs each subcategory
    
    for block in tqdm(selected):
        page_num=block[0]
        rectangle_coords=block[1]
        tag="".join(block[2].split(":")[1:])
        tag=tag.split(".")[0]
        if tag not in look_up: #if tag not in look_up add that tag
            look_up[tag]=[[page_num,rectangle_coords]]
        else:
            look_up[tag].append([page_num,rectangle_coords]) #else append on the same tag
    
    #print(look_up.keys())
    
    for element in copy.deepcopy(look_up).keys():
        if (element not in category_map):
            del look_up[element]
    #print(set(look_up.keys()),len(set(look_up.keys())))

    #get_name("defination1") #this becomes defination


    new_look_up={}

    for k,v in look_up.items():
        name=k
        if(category_map[name] not in new_look_up):
            new_look_up[category_map[name]]=look_up[k]
        else:
            for element in look_up[k]:
                new_look_up[category_map[name]].append(element)

    
    #print(new_look_up)
    #new look ups
    ll=copy.deepcopy(new_look_up)

    #put everything that is not proof in theorem 
    if(binary==True):
        for k,v in ll.items():
            if(k!=category_map["theorem"] and k!= category_map["proof"]):

                for element in new_look_up[k]:
                    new_look_up[category_map["theorem"]].append(element)

                #delete
                del new_look_up[k]

    
    return new_look_up



def convert_yolo(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return x,y,w,h

def boxes_to_yolo(new_look_up,path):
    for k,v in new_look_up.items():
        for i,box in enumerate(v):
            image_no=box[0]

            #read through this image path
            image=path+"/"+"image_"+str(image_no)+'.png'
            im=Image.open(image) 

            w= int(im.size[0]) #the x axis of the image
            h= int(im.size[1]) #the y axis of the image

            xmin=box[1][1][0]
            ymin=box[1][1][1]
            xmax=box[1][2][0]
            ymax=box[1][2][1]

            #print(xmin, xmax, ymin, ymax)

            b = (xmin, xmax, ymin, ymax)

            x,y,w,h = convert_yolo((w,h), b)


            new_look_up[k][i][1]=[x,y,w,h]
            
    return new_look_up

new_look_up=make_lookups(box_cc,category_map,binary=True)
new_look_up=boxes_to_yolo(new_look_up,path)

from pdf2image import convert_from_path
from math import floor
from bs4 import BeautifulSoup as bs
import os
import cv2
from tqdm import tqdm

#wrapping everything together
sample_pdf="/Users/mv96/Desktop/dummy/pdf/toto/toto.pdf" #this is the pdf with links
xml_main="/Users/mv96/Desktop/dummy/xml/toto/toto.xml" #this contains the page locations
annot_xml="/Users/mv96/Desktop/dummy/xml/toto/toto_annot.xml" #this contains annotations width

#converts the pdf into images
all_images,resolution=pdf2_image(sample_pdf,xml_main)

#get magnifications
magnifications=get_magnifications(sample_pdf,resolution)

#get all boxes from annot xml
boxes=get_all_boxes(annot_xml,magnifications)

#get the sample pdf path 
path=sample_pdf.rsplit("/",1)[0]+"/images"


#we have seen 4 methods to filter these patches

#1 works on the pytesseract ocr way it takes time
#2 works on detecting the contours in the image 
#3 works on the area based filtering of patches
#4 works on text extraction by xml


# but text extraction works on box coordinates not on patches
boxes_text=box_to_text_using_xml(xml_main,magnifications,boxes,show_text=False) 

#draw rectangles also gets patches
patches,boxes_per_page=draw_rectangles(path,boxes,show_images=False) #patches only contain the protions of the 

#filter thresholds function uses the 4 functions to filter the boxes to remove noise
area_th=2 #small blocks have low area
contour_th=2 #plain horizontal lines have low conoturs
text_th_ocr=2 #page number boxes have have low text
text_th_xml=3 #xml text below this chars text blocks are ignored

thresholds=[text_th_ocr,contour_th,area_th,text_th_xml] #this is the thresholds list

print("="*20)
#we can filter the boxes based on 4 thresholds defined above
selected,box_cc=filter_thresholds(thresholds,patches,boxes_text,boxes,show_text=False,show_image=False)



patches,boxes_per_page=draw_rectangles(path,box_cc,show_images=False) 
#print(boxes_per_page)


#we now have the cropped images in patches and box per page is the dictionary
try:
    delete_irrelevant_images(selected,all_images) #delete images that are not relevant 
except:
    print("one or more of images not present !")


#get categories
category_map=get_original_labels()


#get the transformed look up tabel
new_look_up=make_lookups(box_cc,category_map,binary=True) #binary denotes we only want proofs and theorems


#convert these boxes into boxes to yolo
new_look_up=boxes_to_yolo(new_look_up)

#write the files to the directory
write_file_directory(category_map,path,new_look_up)


In [ ]:
import numpy as np

In [ ]:
def merge_boxes(box_cc):
    
    #get unique page numbers in a list
    page_coordinate={}
    for element in box_cc:
        if(element[0] not in page_coordinate):
            page_coordinate[element[0]]=[element[1:]]
        else:
            page_coordinate[element[0]].append(element[1:])
            
    uniques_per_page={}
    
    
    for k,v in page_coordinate.items():
        page=k
        values_for_that_page=v
        
        tags_cc={}
        
        for rectangles in values_for_that_page:
            if(rectangles[1] not in tags_cc):
                tags_cc[rectangles[1]]=[rectangles[0]]
            else:
                tags_cc[rectangles[1]].append(rectangles[0])
                
        
        for key,element in zip(tags_cc,tags_cc.values()):
            d=list(map(lambda x:x[0],element)) #bot left ####
            a=list(map(lambda x:x[1],element)) #top left #
            c=list(map(lambda x:x[2],element)) #bot rights
            b=list(map(lambda x:x[3],element)) #top right  #####
            
            c=np.array(c)
            a=np.array(a)
            d=np.array(d)
            b=np.array(b)
            
            
            xmax, ymax = c.max(axis=0)
            c=[xmax,ymax] ###converted c
            
            
            xmin, ymin = a.min(axis=0)
            a=[xmin,ymin] #converted a
            
            #for b we need to apply max min
            
            bmaxx, bmaxy = b.max(axis=0)
            bminx, bminy = b.min(axis=0)
            
            b=[bmaxx, bminy]
            
            
            #for d we need to apply min max
            
            dmaxx, dmaxy = d.max(axis=0)
            dminx, dminy = d.min(axis=0)
            
            d=[dminx, dmaxy]
            
            #after the order is c,a,b,d
            big_box=[d,a,c,b]
            
            tags_cc[key]=big_box
        
        uniques_per_page[k]=tags_cc
    
    final=[]
    for page,boxes in uniques_per_page.items():
        for label,box in boxes.items():
            temp=[page,box,label]
            final.append(temp)
    return final

merge_boxes(box_cc)
    

In [ ]:
#write in file directory

def write_file_directory(category_map,ppath,new_look_up):
    
    #we need this import inside path because path is now a keyword
    from os import path
    
    #make the classes.txt file
    if(path.exists(ppath+"/"+"classes.txt")):
        os.remove(ppath+"/"+"classes.txt")
        
    classes_text=open(ppath+"/"+"classes.txt","w+")
    
    for element in category_map:
        classes_text.write(element+"\n")
        
    classes_text.close()
    
    
    for k,v in new_look_up.items():
        for box in v:
            page_no=box[0]
            values=box[1]
            values=list(map(lambda x:str(x),values))
            values=str(k)+" "+" ".join(values)
            
            #open the file
            f_no=ppath+"/"+"image_"+str(page_no)+".txt"
            
            if(path.exists(f_no)):
                os.remove(f_no)
                
    #print("this is my new look up ",new_look_up)
    
    #make the other files
    for k,v in new_look_up.items():
        for box in v:
            page_no=box[0]
            values=box[1]
            values=list(map(lambda x:str(x),values))
            values=str(k)+" "+" ".join(values)
            
            #open the file
            f_no=ppath+"/"+"image_"+str(page_no)+".txt"
            
            if(path.exists(f_no)):
                file=open(f_no,"a+")
                file.write(values+"\n")
                file.close()
            else:
                file=open(f_no,"w+")
                file.write(values+"\n")
                file.close()
    
    #make the

write_file_directory(category_map,path,new_look_up)

In [ ]:
%%time
from pdf2image import convert_from_path
from math import floor
from bs4 import BeautifulSoup as bs
import os
import cv2
from tqdm import tqdm
import shutil



def get_annotaions_for_yolo(sample_pdf,xml_main,annot_xml):
    
    try:
        #get the sample pdf path 
        path=sample_pdf.rsplit("/",1)[0]+"/images"
        #print(path)
        
        if(os.path.exists(path)):
            shutil.rmtree(path, ignore_errors=True)
        
        #print("this is my sample_pdf",sample_pdf)
        
        remove_pdf_links(sample_pdf)
        #print("done 1")
        
        #converts the pdf into images
        all_images,resolution,skip= pdf2_image(sample_pdf,xml_main)
        #print(skip)
        #print("done 2")

        #get magnifications
        magnifications=get_magnifications(sample_pdf,resolution,all_images)
        #print("done 3")
        
        #get all boxes from annot xml
        boxes=get_all_boxes(annot_xml,magnifications,skip)
        #print("done 4")
        
        #we have seen 4 methods to filter these patches

        #1 works on the pytesseract ocr way it takes time
        #2 works on detecting the contours in the image 
        #3 works on the area based filtering of patches
        #4 works on text extraction by xml


        # but text extraction works on box coordinates not on patches
        boxes_text=box_to_text_using_xml(xml_main,magnifications,boxes,show_text=False,skip=skip)
        #print("done 5")

        #draw rectangles also gets patches
        patches,boxes_per_page=draw_rectangles(path,boxes,show_images=False) #patches only contain the protions of the 
        #print("done 6")
        

        #we can filter the boxes based on 4 thresholds defined above
        selected,box_cc=filter_thresholds(patches,boxes_text,boxes,show_text=False,show_image=False)
        
        box_cc=merge_boxes(box_cc)

        #selected is the text per page
        #print("="*20)
        #print(selected)
        #print("="*20)
        #print("done 7")

        #patches,boxes_per_page=draw_rectangles(path,box_cc,show_images=False) 
        #print("done 8")

        #we now have the cropped images in patches and box per page is the dictionary
        try:
            delete_irrelevant_images(selected,all_images,path) #delete images that are not relevant 
        except:
            print("one or more of images not present !")
        
        #print("done 9")
        

        #get categories
        category_map=get_original_labels()
        
        #print("="*20)
        #print(box_cc) #this is the box coordinate and the label (dict)
        #print("="*20)
        #print("done 10")
        
        #get the transformed look up table
        new_look_up=make_lookups(box_cc,category_map,binary=True) #binary denotes we only want proofs and theorems
        #print("done 11")
        
        
        #convert these boxes into boxes to yolo
        new_look_up=boxes_to_yolo(new_look_up,path)
        #new_look_up=boxes_to_pascal_voc(new_look_up,path)
        #print(new_look_up)
        #print(new_look_up.keys())
        #print("done 12")

        #write the files to the directory
        write_file_directory(category_map,path,new_look_up)
        #print("done 13")
        print("success:",sample_pdf)
        return 0
        
    except FileNotFoundError:
        if(not os.path.exists(xml_main)):
            print("one of the files missing, please check the folder : {}".format(xml_main))
            
        return 1
    except:
        return 2
    
#check /Users/mv96/tkb-pdfs/1901.09434

sample_pdf='' #this is the pdf with links
xml_main='' #this contains the page locations
annot_xml='' #this contains annotations width


get_annotaions_for_yolo(sample_pdf,xml_main,annot_xml)    

In [ ]:
pdf_directory_6000="" #the name of the directory containing all the 6000 pdfs

import os
arr = os.listdir(pdf_directory_6000)


#go in each folder
from os import walk

#getting the file names of xml files and pdf's
paths=[]
for (dirpath, dirnames, filenames) in walk(pdf_directory_6000):
    for sub_folder in dirnames:
        for (dirpath, dirnames, filenames) in walk(pdf_directory_6000+"/"+sub_folder):
            pdf_path=pdf_directory_6000+"/"+sub_folder+"/{}.pdf".format(sub_folder)
            xml_main=pdf_directory_6000+"/"+sub_folder+"/{}.xml".format(sub_folder)
            annot_xml=pdf_directory_6000+"/"+sub_folder+"/{}_annot.xml".format(sub_folder)
            if([pdf_path,xml_main,annot_xml] not in paths):
                paths.append([pdf_path,xml_main,annot_xml])




In [ ]:
#one by one execution

def logs_write(paths,flags):
    logs=list(map(lambda x:x[0].rsplit("/",1)[0],paths))
    logs_dict={}
    
    for element,label in zip(logs,flags):
        if(label not in logs_dict):
            logs_dict[label]=[element]
        else:
            logs_dict[label].append(element)
            
    return logs_dict



flags=[]

one_by_one=False; thresh=len(paths); #for all use len(paths)

paths=paths[:thresh+1]

if(one_by_one==True): #one by one execution will label data one at a time
    
    for file in tqdm(paths):
        ret=get_annotaions_for_yolo(file[0],file[1],file[2])
        flags.append(ret)
        
    logs_dict=logs_write(paths,flags)


# multi processing the code to further optimize the speed of labelling

In [ ]:
#multi process the code
from joblib import Parallel, delayed  
from tqdm import tqdm

#there could be over flow of resources so one can reduce the n_jobs parameter
res = Parallel(n_jobs=-1, verbose=10)(delayed(get_annotaions_for_yolo)(file[0],file[1],file[2])
                             for file in tqdm(paths))



print(res)


In [ ]:
logs=logs_write(paths,res)
print(logs)

In [ ]:
#list all the folders that are therein 

In [ ]:
#merging the images in the dataset
pdf_directory_6000="" #the name of the directory containing all the 6000 pdfs

#traverse through each of the directory and print the files
#we have 4399 folders with title images
#we have 4161 folders that have images
#we have 3769 folders that actually have images and text associated

count=0
selected=[]
for element in logs[None]: #only the successful logs directory go through this
    dir_list = os.listdir(element) 
    
    
    if("images" in dir_list): #
        dir_list_1=os.listdir(element+"/"+"images") 
        
        images_list=list(filter(lambda x: x.endswith(".png"),dir_list_1)) #select all png
        text_list=list(filter(lambda x: x.endswith(".txt"),dir_list_1)) #select all txt
        
        #print(images_list)
        #print(text_list)
        
        for img in images_list:
            just_name=img.split(".")[0]
            
            if(just_name+".txt" in text_list):
                selected.append(element+"/"+"images"+"/"+img)
                selected.append(element+"/"+"images"+"/"+just_name+".txt")
                
#this contains all the successful folders with images that have images and corresponding .txt
print(len(selected))    #images and text in one big list

images_list=list(filter(lambda x: x.endswith(".png"),selected)) #image_list
text_list=list(filter(lambda x: x.endswith(".txt"),selected)) #txt_list

In [ ]:
#example to copy and paste file on OS
from tqdm import tqdm
from shutil import copyfile
#copyfile("/Users/mv96/Desktop/me_image.jpeg", "/Users/mv96/Desktop/urgent_things/me_image.jpeg")

In [ ]:
# we wanna save everything into a new directory
#train_directory="/home/mishra/tkb-images"
from shutil import copyfile
import os

dest="" #the directory where you want to save the images

if(os.path.exists(dest)):
    print("path exists !!!")
else:
    print("create a new directory of tkb images")
    os.mkdir(dest)


for img,text in zip(images_list,text_list):

    image_path_write=dest+"_".join(img.rsplit("/",3)[1:])

    text_path_write=dest+"_".join(text.rsplit("/",3)[1:])
    
    
    copyfile(img, image_path_write)
    copyfile(text, text_path_write)


 51921 images saved in the directory

1. copy classes.txt from any of the one file to this to make sure all class labels are present 

2. make sure the images are easily opened in labelimg format to just be sure of the op directory

3. follow yolo v4 collab notebook https://colab.research.google.com/drive/1FnjVjnwaPSnX6tZSwxBm4w13-CmnDcEt?usp=sharing and tutorial on youtube https://www.youtube.com/watch?v=mmj3nxGT2YQ&t=1143s

# Class based implementation

In [ ]:
from math import floor
from pdf2image import convert_from_path
from math import floor
from bs4 import BeautifulSoup as bs
import cv2
from tqdm import tqdm
import natsort 
from PIL import Image
from pytesseract import image_to_string
import copy
from itertools import compress
import subprocess
import os
import numpy as np
import shutil

In [ ]:
class Yolo_Pre_Processing:
    
    out_pdf=""
    
    def __init__(self):
        
        #default values
        self.sample_pdf='/Users/mv96/Desktop/temp/1902.11202/1902.11202.pdf' 
        self.xml_main='/Users/mv96/Desktop/temp/1902.11202/1902.11202.xml' 
        self.annot_xml='/Users/mv96/Desktop/temp/1902.11202/1902.11202_annot.xml' 
        self.path=self.sample_pdf.rsplit("/",1)[0]+"/images"
        self.area_th=2 #small blocks have low area
        self.contour_th=8 #plain horizontal lines have low contours
        self.text_th_ocr=2 #page number boxes have have low text
        self.text_th_xml=3
        self.thresholds=[self.text_th_ocr,self.contour_th,self.area_th,self.text_th_xml]
        self.pdf_directory="/Users/mv96/tkb_pdfs"
        self.flags=[]
        

    def remove_pdf_links(self,out_pdf=None):

        if(out_pdf==None):
            out_pdf=self.sample_pdf.rsplit(".",1)
            out_pdf_temp=out_pdf[0]+"_stripped_temp."+out_pdf[1]
            out_pdf=out_pdf[0]+"_stripped."+out_pdf[1]



        #code for conversion
        first_cmd=" ".join(["pdftk", "{}".format(self.sample_pdf),"output", "{}".format(out_pdf_temp), "uncompress"])
        second_cmd="LC_ALL=C sed -n '/^\/Annots/!p' {} > {}".format(out_pdf_temp,self.sample_pdf)

        #call first command 

        subprocess.call(first_cmd,shell=True)

        #call second command
        ret=subprocess.call(second_cmd,shell=True)

        os.remove(out_pdf_temp)
        

        
    def pdf2_image(self):
    
        #get the resolution of each pdf page from xml
        content = []
        # Read the XML file
        with open(self.xml_main, "r") as file:
            # Read each line in the file, readlines() returns a list of lines
            content = file.readlines()
            # Combine the lines in the list into a string
            content = "".join(content)
            bs_content = bs(content, "xml")
            #print(bs_content)
            page_info=bs_content.find_all("Page")
            resolution=[]
            for page in page_info:
                height=floor(float(page.get("HEIGHT")))
                width=floor(float(page.get("WIDTH")))
                resolution.append((width,height))
        file.close()

        #print(resolution)

        #convert the pdf to image using the coordinates from the xml_file
        all_pages=[]

        ff=1
        lf=len(resolution)
        image = convert_from_path(self.sample_pdf,first_page=ff,last_page=lf,use_pdftocairo=True,fmt="png")

        skip=False #default value

        if(len(resolution)!=len(image)):
            skip=True
            resolution.pop(0)


        #print(all_pages)

        #make a new directory in pdf folder
        self.resolution=resolution
        self.skip=skip
        self.all_images=image
        

        return image,resolution,skip
    
    def get_magnifications(self):
    
        try:
            path=self.sample_pdf.rsplit("/",1)[0]+"/images"
            #print(path)
            os.mkdir(path)
        except:
            pass
            #print("path already exists !!!!")

        #print(path)     
        magnifications=[]

        i=0
        for image,res in zip(self.all_images,self.resolution):
            #print (image.format, image.size, image.mode)
            new_width=image.size[0]
            original_width=res[0]
            new_height=image.size[1]
            original_height=res[1]
            mag_width=new_width/original_width
            mag_height=new_height/original_height
            magnifications.append([mag_width,mag_height])
            #image.show()
            image.save(path+"/"+"image_"+str(i+1)+'.png', 'PNG')
            i+=1
        
        self.magnifications=magnifications
        

        return magnifications
    
    
    def get_all_boxes(self):
        # Read the XML file
        with open(self.annot_xml, "r") as file:
            # Read each line in the file, readlines() returns a list of lines
            content = file.readlines()
            # Combine the lines in the list into a string
            #print(content)
            content = "".join(content)
            bs_content = bs(content, "xml")


            page_info=bs_content.find_all("ANNOTATION")
            quads=bs_content.find_all("QUADRILATERAL")



            extractions=[]
            #filter magnifications for certain pages


            for action_,quad in zip(page_info,quads):
                if(action_.DEST.string!=None): #valid box
                    page_no=int(action_.get("pagenum")) #basically the page_num

                    class_=action_.DEST.string #the type of the box

                    #if the type of the box is some link then skip it dont extract it
                    if("@" in class_):
                        continue


                    coord=[[floor(float(x.get("HPOS"))),floor(float(x.get("VPOS")))] for x in quad] #the coordinates
                    if(self.skip):
                        extractions.append([page_no-1,coord,class_]) 
                    else:
                        extractions.append([page_no,coord,class_]) 

        file.close()

        #print(extractions)
        #we need to filter magnifications for each page
        for i in range(len(self.magnifications)):
            #print(i)
            for k,elements in enumerate(extractions):
                #print(elements)
                #print(elements)
                if(elements[0]==i+1):

                    #print(extractions[k][1])

                    #change the two coordinates
                    for d in range(len(elements[1])): #for all coordinates change their pix values
                        extractions[k][1][d][0]=floor(extractions[k][1][d][0]*self.magnifications[i][0])
                        extractions[k][1][d][1]=floor(extractions[k][1][d][1]*self.magnifications[i][1])

        
        
        self.boxes=extractions
        return extractions
    
    def draw_rectangles(self,show_images=False):
        """takes the images from the image path reads them and plots a rectange on them
        also return the patches for each detected page  
        """
        #print(images_dir)
        
        pages=[]
        box=[]
        names=[]

        boxes_per_page={}
        for info in self.boxes:
            page=info[0]
            rect=info[1]
            if(page not in boxes_per_page):
                boxes_per_page[page]=[rect]
            else:
                boxes_per_page[page].append(rect)

        #print(boxes_per_page)
        patches={}

        for k,v in boxes_per_page.items():
            image_path=self.path+f"/image_{k}.png"
            image=cv2.imread(image_path)


            #v=[[1,2,3,4],[1,2,3,4]]
            cropped_images=[]

            #crop the images
            for boxes in v:
                #print(boxes)
                top_left=boxes[1] #x1,y1
                bottom_right=boxes[2] #x2,y2


                #cropping images
                cropped = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]


                cropped_images.append(cropped)


            patches[k]= cropped_images

            if(show_images): #shows all boxes including filtered boxes
                for boxes in v:
                    (x1,y1),(x2,y2)=boxes[1],boxes[2]
                    image=cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 1)
                    cv2.imshow("test",image)
                    cv2.waitKey()
        
        self.patches=patches
        self.boxes_per_page=boxes_per_page
        
        return patches,boxes_per_page

    def delete_irrelevant_images(self):
        
        
        for i in range(1,len(self.all_images)): #from image 1 to last
            if(i not in self.patches.keys()):
                os.remove(self.path+"/"+"image_"+str(i)+'.png')
        
            
    def image_to_text(self,image,show_image=False,show_text=False):

        # By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
        # we need to convert from BGR to RGB format/mode:
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ocr_chars=image_to_string(img_rgb)

        if(show_text):
            print("tesseract values >>>>>>>")
            print("this is my length: ",len(ocr_chars))
            print(ocr_chars)
            print("tesseract values <<<<<<<")
            print("\n")


        if(show_image):
            cv2.imshow("detection :",numpy_arr_of_image)
            cv2.waitKey()
            cv2.destroyAllWindows()

        return ocr_chars
    
    def get_contours_of_image(self,image,show_text=False,show_image=False):
               
        #we need to convert the image into gray scale to get contour values
        # Grayscale
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        gray = cv2.cvtColor(rgb_image,cv2.COLOR_BGR2GRAY)

        # Find Canny edges
        edged = cv2.Canny(gray, 30, 200)

        #print("contours: >>>>>>>")
        contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if(show_text):
            print("Number of Contours found = " + str(len(contours)))



        if(show_image):
            # Draw all contours
            # Use '-1' as the 3rd parameter to draw all
            cv2.drawContours(image, contours, -1, (0,255,0), 1)
            cv2.imshow('Contours:'+str(len(contours)), image)
            cv2.waitKey()
            cv2.destroyAllWindows()

        return len(contours)
    
    def image_to_area(self,image,show_text=False):
        width=image.shape[0]
        height=image.shape[1]
        area=(width*height)/100 #in pixel square

        if(show_text):
            print(area)

        return area
    
    def box_to_text_using_xml(self,show_text=False,skip=False):
        """take a box coodinates xml file to check and adjusted magnifications
        and return what characters are inside the box basically text version
        """

        with open(self.xml_main,"r") as file:
            content=file.readlines()
            content = "".join(content)
            bs_content = bs(content, "xml")
            #print(bs_content)
            text_blocks={}
            page_info=bs_content.find_all("Page")

            if(self.skip):
                page_info.pop(0)


            for i,page in enumerate(page_info): #traverse through all pages

                page_text=page.find_all("TextLine")
                text=[]
                for text_bl in page_text:

                    for element in text_bl:
                        if(str(element).startswith("<String")): #its a string 
                            try:
                                content=element.get("CONTENT")
                                #print(content)
                                h_pos=floor(float(element.get("HPOS"))*self.magnifications[i][0])
                                v_pos=floor(float(element.get("VPOS"))*self.magnifications[i][1])
                                #we can also extract the font 

                                text.append([content,h_pos,v_pos])
                            except:
                                print(element)
                                print(i)
                                print(self.magnifications[i][0])
                                print(content)


                        if(str(element).startswith("<SP")):
                            val=text.pop()
                            val[0]+=" "
                            text.append(val)

                    #print(text)

                text_blocks[i+1]=text

            file.close()



        box_text={}
        string_cats=[]
        for box in self.boxes: #basically the big box
            all_chars_on_page=text_blocks[box[0]] #small boxes

            #coordinates of the big box
            top_left_bb= copy.deepcopy(box[1][1])
            bot_right_bb=copy.deepcopy(box[1][2])


            #print(all_chars_on_page)
            string_cat=""

            for chars in all_chars_on_page:
                if(top_left_bb[0]<=chars[1]<=bot_right_bb[0] and top_left_bb[1]<=chars[2]<=bot_right_bb[1]):
                    string_cat+=chars[0]

            string_cats.append((box[0],string_cat))

        for element in string_cats:
            if(element[0] not in box_text):
                box_text[element[0]]=[element[1]]
            else:
                box_text[element[0]].append(element[1])
        
        self.box_text=box_text
        return box_text
        
    
    def filter_thresholds(self,show_text=False,show_image=False):
        
        #cleaning the mess
        thresholds=[self.text_th_ocr,self.contour_th,self.area_th,self.text_th_xml]
        patches=self.patches
        boxes_text=self.box_text
        boxes=self.boxes
        
        box_map_cc=[]
        selection={}
        for page_info in patches.keys():

            verdicts=[]

            for image,text_xml in zip(patches[page_info],boxes_text[page_info]):

                area=self.image_to_area(image,show_text=False)
                contours_count=self.get_contours_of_image(image,show_text=False)
                #text_ocr=self.image_to_text(numpy_arr_of_image,show_image=False,show_text=False)

                if(show_text):
                    print("image area: ",area)
                    print("contours count: ",contours_count)
                    #print("text using ocr: and length",(text_ocr,len(text_ocr)))
                    print("text using xml: and length",(text_xml,len(text_xml)))
                    print("=======")

                if(show_image):
                    name="area {}. contours {}. text {}.".format(area,contours_count,text_xml)
                    cv2.imshow("name",image)
                    cv2.waitKey()
                    cv2.destroyAllWindows()

                small_area= area<self.area_th
                small_contour_count= contours_count<self.contour_th

                #less_text_ocr= len(text_ocr)<text_th_ocr #using tesseract ocr
                less_text_xml=len(text_xml)<self.text_th_xml

                less_text=less_text_xml

                verdict=not(small_area or small_contour_count or less_text)
                verdicts.append(verdict)
                box_map_cc.append(verdict)


            selection[page_info]=verdicts


        selection_=copy.deepcopy(selection)


        for k,sels,box in zip(selection_.copy(),selection_.values(),boxes_text.values()):
            new_images=list(compress(box, sels))

            if(len(new_images)==0):
                del selection[k]
            else:
                selection[k]=new_images


        #this is where we will start to filter each of the images
        box_map_cc=list(compress(boxes, box_map_cc))
        
        self.selected=selection
        self.boxes=box_map_cc
        return selection ,box_map_cc
    
    def get_original_labels(self):
        """basically assigning labesls"""
        categories=["assumption","claim","conjecture","corollary","definition","lemma","proof","proposition",
                    "remark", "theorem"]

        category_map={}

        for i,element in enumerate(categories):
            category_map[element]=i
        
        self.category_map=category_map
        return category_map
    
    def make_lookups(self,binary=True):
        look_up={} #this is the orgnal_lookup table matching page vs each subcategory
        

        for block in tqdm(self.boxes):
            page_num=block[0]
            rectangle_coords=block[1]
            tag="".join(block[2].split(":")[1:])
            tag=tag.split(".")[0]
            if tag not in look_up: #if tag not in look_up add that tag
                look_up[tag]=[[page_num,rectangle_coords]]
            else:
                look_up[tag].append([page_num,rectangle_coords]) #else append on the same tag

        #print(look_up.keys())

        for element in copy.deepcopy(look_up).keys():
            if (element not in self.category_map):
                del look_up[element]
        #print(set(look_up.keys()),len(set(look_up.keys())))

        #get_name("defination1") #this becomes defination


        new_look_up={}

        for k,v in look_up.items():
            name=k
            if(self.category_map[name] not in new_look_up):
                new_look_up[self.category_map[name]]=look_up[k]
            else:
                for element in look_up[k]:
                    new_look_up[self.category_map[name]].append(element)


        #print(new_look_up)
        #new look ups
        ll=copy.deepcopy(new_look_up)

        #put everything that is not proof in theorem 
        if(binary==True):
            for k,v in ll.items():
                if(k!=self.category_map["theorem"] and k!= self.category_map["proof"]):

                    for element in new_look_up[k]:
                        new_look_up[self.category_map["theorem"]].append(element)

                    #delete
                    del new_look_up[k]

        self.new_look_up=new_look_up
        return new_look_up



    def convert_yolo(self,size, box):
        dw = 1./size[0]
        dh = 1./size[1]
        x = (box[0] + box[1])/2.0
        y = (box[2] + box[3])/2.0
        w = box[1] - box[0]
        h = box[3] - box[2]
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
        return x,y,w,h

    def boxes_to_yolo(self):
        
        for k,v in self.new_look_up.items():
            for i,box in enumerate(v):
                image_no=box[0]

                #read through this image path
                image=self.path+"/"+"image_"+str(image_no)+'.png'
                im=Image.open(image) 

                w= int(im.size[0]) #the x axis of the image
                h= int(im.size[1]) #the y axis of the image

                xmin=box[1][1][0]
                ymin=box[1][1][1]
                xmax=box[1][2][0]
                ymax=box[1][2][1]

                #print(xmin, xmax, ymin, ymax)

                b = (xmin, xmax, ymin, ymax)

                x,y,w,h = self.convert_yolo((w,h), b)


                self.new_look_up[k][i][1]=[x,y,w,h]
        
        return self.new_look_up
    
    
    def merge_boxes(self):
    
        #get unique page numbers in a list
        page_coordinate={}
        for element in self.boxes:
            if(element[0] not in page_coordinate):
                page_coordinate[element[0]]=[element[1:]]
            else:
                page_coordinate[element[0]].append(element[1:])

        uniques_per_page={}


        for k,v in page_coordinate.items():
            page=k
            values_for_that_page=v

            tags_cc={}

            for rectangles in values_for_that_page:
                if(rectangles[1] not in tags_cc):
                    tags_cc[rectangles[1]]=[rectangles[0]]
                else:
                    tags_cc[rectangles[1]].append(rectangles[0])


            for key,element in zip(tags_cc,tags_cc.values()):
                d=list(map(lambda x:x[0],element)) #bot left ####
                a=list(map(lambda x:x[1],element)) #top left #
                c=list(map(lambda x:x[2],element)) #bot rights
                b=list(map(lambda x:x[3],element)) #top right  #####

                c=np.array(c)
                a=np.array(a)
                d=np.array(d)
                b=np.array(b)


                xmax, ymax = c.max(axis=0)
                c=[xmax,ymax] ###converted c


                xmin, ymin = a.min(axis=0)
                a=[xmin,ymin] #converted a

                #for b we need to apply max min

                bmaxx, bmaxy = b.max(axis=0)
                bminx, bminy = b.min(axis=0)

                b=[bmaxx, bminy]


                #for d we need to apply min max

                dmaxx, dmaxy = d.max(axis=0)
                dminx, dminy = d.min(axis=0)

                d=[dminx, dmaxy]

                #after the order is c,a,b,d
                big_box=[d,a,c,b]

                tags_cc[key]=big_box

            uniques_per_page[k]=tags_cc

        final=[]
        for page,boxes in uniques_per_page.items():
            for label,box in boxes.items():
                temp=[page,box,label]
                final.append(temp)
                
        self.boxes=final
        return final
    
    def write_file_directory(self):
    
        #we need this import inside path because path is now a keyword
        from os import path

        #make the classes.txt file
        if(path.exists(self.path+"/"+"classes.txt")):
            os.remove(self.path+"/"+"classes.txt")

        classes_text=open(self.path+"/"+"classes.txt","w+")

        for element in self.category_map:
            classes_text.write(element+"\n")

        classes_text.close()


        for k,v in self.new_look_up.items():
            for box in v:
                page_no=box[0]
                values=box[1]
                values=list(map(lambda x:str(x),values))
                values=str(k)+" "+" ".join(values)

                #open the file
                f_no=self.path+"/"+"image_"+str(page_no)+".txt"

                if(path.exists(f_no)):
                    os.remove(f_no)

        #print("this is my new look up ",new_look_up)

        #make the other files
        for k,v in self.new_look_up.items():
            for box in v:
                page_no=box[0]
                values=box[1]
                values=list(map(lambda x:str(x),values))
                values=str(k)+" "+" ".join(values)

                #open the file
                f_no=self.path+"/"+"image_"+str(page_no)+".txt"

                if(path.exists(f_no)):
                    file=open(f_no,"a+")
                    file.write(values+"\n")
                    file.close()
                else:
                    file=open(f_no,"w+")
                    file.write(values+"\n")
                    file.close()
                    
    def get_annotaions_for_yolo(self):
    
        try:
            
            if(os.path.exists(self.path)):
                shutil.rmtree(self.path, ignore_errors=True)

            #print("this is my sample_pdf",sample_pdf)
            self.remove_pdf_links()
            self.pdf2_image()
            self.get_magnifications()
            self.get_all_boxes()
            self.draw_rectangles()
            self.delete_irrelevant_images()
            self.box_to_text_using_xml()
            self.filter_thresholds()
            self.merge_boxes() #we can disable this if we dont want to merge inputs
            self.get_original_labels()
            self.make_lookups()
            self.boxes_to_yolo()
            self.write_file_directory()
            
            print("success:",self.sample_pdf)
            return 0

        except FileNotFoundError:
            if(not os.path.exists(self.xml_main)):
                print("one of the files missing, please check the folder : {}".format(self.xml_main))
            return 1
        except:
            return 2
        
        
    def pdfs_from_directory(self):    
        
        arr = os.listdir(self.pdf_directory)

        #getting the file names of xml files and pdf's
        paths=[]
        for (dirpath, dirnames, filenames) in os.walk(self.pdf_directory):
            for sub_folder in dirnames:
                for (dirpath, dirnames, filenames) in os.walk(self.pdf_directory+"/"+sub_folder):
                    pdf_path=self.pdf_directory+"/"+sub_folder+"/{}.pdf".format(sub_folder)
                    xml_main=self.pdf_directory+"/"+sub_folder+"/{}.xml".format(sub_folder)
                    annot_xml=self.pdf_directory+"/"+sub_folder+"/{}_annot.xml".format(sub_folder)
                    if([pdf_path,xml_main,annot_xml] not in paths):
                        paths.append([pdf_path,xml_main,annot_xml])
                        
        self.paths=paths
        return paths
        
    def logs_write(self):
        logs=list(map(lambda x:x[0].rsplit("/",1)[0],self.paths))
        logs_dict={}

        for element,label in zip(logs,self.flags):
            if(label not in logs_dict):
                logs_dict[label]=[element]
            else:
                logs_dict[label].append(element)
        
        self.logs_dict=logs_dict
        

In [ ]:
test=Yolo_Pre_Processing()
test.remove_pdf_links()
test.pdf2_image()
test.get_magnifications()
test.get_all_boxes()
test.draw_rectangles()
test.delete_irrelevant_images()
test.box_to_text_using_xml()
test.filter_thresholds()
#test.merge_boxes() #we can disable this if we dont want to merge certain text lines
test.get_original_labels()
test.make_lookups()
test.boxes_to_yolo()
test.write_file_directory()